In [18]:
import sys
import os
import numpy as np 
from keras.utils import to_categorical
from validation import compute_f1
from keras.models import Model
from keras.layers import TimeDistributed,Conv1D,Dense,Embedding,Input,Dropout,LSTM,Bidirectional,MaxPooling1D,Flatten,concatenate
# from prepro import readfile,createBatches,createMatrices,iterate_minibatches,addCharInformatioin,padding
from keras.utils import plot_model,Progbar
from keras.preprocessing.sequence import pad_sequences
from keras.initializers import RandomUniform
from keras.optimizers import Adam
import keras.backend as K
from sklearn.metrics import f1_score
from keras.callbacks import ModelCheckpoint, Callback
from keras_contrib.layers import CRF
from keras.utils import Sequence
from numpy import newaxis
from random import shuffle
import math
import sklearn
import subprocess
import fastText
import pickle

In [2]:
def getCasing(word, caseLookup):
    
    if word == 'PADDING_TOKEN':
        return(caseLookup['PADDING_TOKEN'])
    
    casing = 'other'
    
    numDigits = 0
    for char in word:
        if char.isdigit():
            numDigits += 1
            
    digitFraction = numDigits / float(len(word))
    
    if word.isdigit(): #Is a digit
        casing = 'numeric'
    elif digitFraction > 0.5:
        casing = 'mainly_numeric'
    elif word.islower(): #All lower case
        casing = 'allLower'
    elif word.isupper(): #All upper case
        casing = 'allUpper'
    elif word[0].isupper(): #is a title, initial char upper, then all lower
        casing = 'initialUpper'
    elif numDigits > 0:
        casing = 'contains_digit'
   
    return caseLookup[casing]

In [46]:
def get_sentences_wiki(path):
    sentences = []
    with open(path, 'r', encoding = 'UTF-8') as f:
        sentence = []
        for line in f:
            
            line = line.strip()
            
            # append sentence
            if len(line) == 0:
                if len(sentence):
                    sentences.append(sentence[:max_sequence_length])
                sentence = []
                continue
            
            # get sentence tokens
            splits = line.split()
            if splits[0] == '#':
                continue
            if len(splits) != 3:
                # print(line)
                continue
            temp = [splits[1],splits[2]]
            sentence.append(temp)
        
        # append last
        if len(sentence):
            sentences.append(sentence)    
    return sentences

In [47]:
# Load label mapping
import json
indexMappings = json.load(open("../models/final_model_germeval.indexes", "r"))
idx2Label = {int(k):v for k,v in indexMappings[0].items()}
label2Idx = indexMappings[1]
char2Idx = indexMappings[2]
case2Idx = indexMappings[3]

In [49]:
max_sequence_length = 56

In [48]:
trainSentences = get_sentences_wiki('../data/weakly_supervised_ner_wiki.train')
devSentences = get_sentences_wiki('../data/weakly_supervised_ner_wiki.dev')

print(len(trainSentences))
print(len(devSentences))

KeyboardInterrupt: 

In [52]:
count = 0
for i, s in enumerate(trainSentences):
    for t in s:
        if len(s) > max_sequence_length:
            print(s)

In [50]:
# POS ALTERNATIVE

trainSentences = get_sentences_wiki('../data/weakly_supervised_pos_wiki.train')
print(len(trainSentences))
devSentences = trainSentences[1400000:]
trainSentences = trainSentences[:1400000]

#devSentences = trainSentences[1000:1200]
#trainSentences = trainSentences[:1000]

print(len(trainSentences))
print(len(devSentences))

1471498
1400000
71498


In [51]:
labelSet = set()
for dataset in [trainSentences, devSentences]:
    for sentence in dataset:
        for word, label in sentence:
            labelSet.add(label)

label2Idx = {"PADDING_TOKEN":0}
for label in labelSet:
    label2Idx[label] = len(label2Idx)
idx2Label = {v: k for k, v in label2Idx.items()}
print(len(label2Idx))
print(idx2Label)

55
{0: 'PADDING_TOKEN', 1: 'PDAT', 2: '$(', 3: 'APPR', 4: 'VAINF', 5: 'PWAV', 6: 'PWAT', 7: 'VMPP', 8: 'PWS', 9: 'NNE', 10: 'VVIMP', 11: 'FM', 12: 'APZR', 13: 'KON', 14: 'PDS', 15: 'PIAT', 16: 'TRUNC', 17: 'VVIZU', 18: 'PIS', 19: '$.', 20: 'VAPP', 21: 'ART', 22: 'PTKANT', 23: 'VVINF', 24: 'NN', 25: 'PTKNEG', 26: 'PPOSS', 27: 'NE', 28: 'PTKVZ', 29: 'VAIMP', 30: 'VVFIN', 31: 'PRF', 32: 'KOKOM', 33: 'PRELAT', 34: 'ADV', 35: 'XY', 36: 'APPO', 37: 'PPER', 38: 'KOUS', 39: 'VAFIN', 40: 'CARD', 41: 'KOUI', 42: 'VMINF', 43: 'ADJA', 44: 'ITJ', 45: 'VVPP', 46: 'PROAV', 47: 'VMFIN', 48: '$,', 49: 'ADJD', 50: 'PTKA', 51: 'PRELS', 52: 'APPRART', 53: 'PPOSAT', 54: 'PTKZU'}


In [28]:
# del trainSentences[1096633]

In [29]:
print(trainSentences[0])

[['K-525', 'NN'], ['Archangelsk', 'NE'], ['K-525', 'NE'], ['wurde', 'VAFIN'], ['am', 'APPRART'], ['25', 'ADJA'], ['.', '$('], ['Juli', 'NN'], ['1975', 'CARD'], ['auf', 'APPR'], ['Kiel', 'NE'], ['gelegt', 'VVPP'], ['.', '$.']]


In [31]:
caseEmbeddings = np.identity(len(case2Idx), dtype='float32')

In [32]:
print(caseEmbeddings)
print(case2Idx)

[[1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1.]]
{'contains_digit': 7, 'PADDING_TOKEN': 0, 'numeric': 1, 'initialUpper': 4, 'allUpper': 3, 'allLower': 2, 'other': 5, 'mainly_numeric': 6}


In [33]:
print(char2Idx)

{'а': 299, '傳': 2, 'X': 9, 'O': 11, 'Е': 258, 'ū': 98, 'ę': 99, 'ö': 116, 'Œ': 78, 'N': 157, 'β': 247, '妃': 149, '„': 233, '‚': 146, 'æ': 124, '[': 54, 'ɨ': 10, '佐': 232, 'ő': 112, 'π': 148, '→': 323, 'ī': 227, 'V': 319, '=': 110, 'ψ': 128, '*': 52, '<': 123, '\x9a': 61, 'G': 256, 'ú': 53, 'É': 143, 'ø': 314, '§': 283, 'ł': 70, 'ض': 210, '$': 43, 'ラ': 310, '\x80': 23, 'â': 100, 'B': 33, 'Т': 260, 'p': 46, 'р': 21, 'o': 79, 'à': 291, '_': 29, '?': 178, '>': 94, '대': 63, 'ã': 167, '太': 264, '殿': 102, 'Ü': 203, '0': 269, 'ḳ': 159, 'Å': 104, 'ë': 280, 'ž': 274, 'Y': 37, '}': 45, '別': 156, 'I': 130, 'č': 152, '¹': 281, '公': 31, 'Ä': 140, ')': 229, 'Š': 235, 'ν': 158, 'ь': 7, 'С': 253, 'q': 321, '-': 244, 'φ': 292, '冲': 27, '−': 164, 'î': 114, '루': 306, 'ğ': 76, 'т': 238, 'W': 301, '東': 286, 'r': 320, 'ʻ': 162, 'ç': 251, 'é': 313, 'm': 139, 'В': 295, 'ô': 191, 'ć': 16, '£': 243, '鷹': 60, '동': 107, 'х': 198, '‘': 145, 'σ': 257, 'd': 40, 'П': 300, '@': 207, 'ـ': 135, '·': 115, 'у': 4, ',': 200

In [34]:
print(devSentences[0]) 

[['Die', 'ART'], ['Kerr-Newman-Metrik', 'NN'], ['ist', 'VAFIN'], ['eine', 'ART'], ['exakte', 'ADJA'], ['Lösung', 'NN'], ['für', 'APPR'], ['sowohl', 'KON'], ['rotierende', 'ADJA'], ['als', 'KON'], ['auch', 'ADV'], ['elektrisch', 'ADJD'], ['geladene', 'ADJA'], ['schwarze', 'ADJA'], ['Löcher', 'NN'], ['in', 'APPR'], ['vier', 'CARD'], ['Dimensionen', 'NN'], ['.', '$.']]


In [14]:
ft = fastText.load_model("../embeddings/wiki.de.bin")
# ft = fastText.load_model("../embeddings/cc.de.300.bin")

nb_embedding_dims = ft.get_dimension()

In [35]:
print(nb_embedding_dims)
print(len(trainSentences[0]))

300
13


In [36]:
def get_label_from_categorical(a):
    labels = []
    for label in a:
        label = np.ndarray.tolist(label)
        label = np.argmax(label)
        labels.append(label)
    return(labels)

def predict_sequences(sentences):
    steps = 0
    true_labels = []
    pred_labels = []
    all_true_labels = []
    for s in sentences:
        all_true_labels.append([w[1] for w in s])
    all_pred_labels = model.predict_generator(NerSequence(sentences))
    
    for s_id, s in enumerate(all_true_labels):
        not_padded_true = []
        not_padded_pred = []
        predicted_labels = get_label_from_categorical(all_pred_labels[s_id])
        for t_id, t in enumerate(s):
            if t != 'PADDING_TOKEN': # skip PADDING_TOKEN 
                not_padded_true.append(label2Idx[t])
                not_padded_pred.append(predicted_labels[t_id])
        true_labels.append(not_padded_true)
        pred_labels.append(not_padded_pred)
    
    return(true_labels, pred_labels)

In [37]:
class NerSequence(Sequence):
    def __init__(self, sentence_data, shuffle_data = False, batch_size=32):
        self.sentence_data = sentence_data
        self.shuffle_data = shuffle_data
        self.batch_size = batch_size
        self.indices = np.arange(len(self.sentence_data))
        if self.shuffle_data:
            np.random.shuffle(self.indices)

    def __len__(self):
        return math.ceil(len(self.sentence_data) / self.batch_size)

    def __getitem__(self, idx):
        inds = self.indices[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_x, batch_y = self.get_processed_data(inds)
        return batch_x, batch_y
    
    def on_epoch_end(self):
        if self.shuffle_data:
            np.random.shuffle(self.indices)
        
    def get_processed_data(self, inds):
        word_embeddings = []
        case_embeddings = []
        char_embeddings = []
        
        output_labels = []

        for index in inds: 
            sentence = self.sentence_data[index]

            temp_word= []
            temp_casing = []
            temp_char= []

            temp_output=[]

            # padding
            words_to_pad = max_sequence_length - len(sentence)
            for i in range(words_to_pad):
                sentence.append(['PADDING_TOKEN', 'PADDING_TOKEN'])

            # create data input for words
            for word in sentence:
                word, label = word
                temp_output.append(label2Idx[label])

                casing = getCasing(word, case2Idx)
                temp_casing.append(casing)

                if word == 'PADDING_TOKEN':
                    temp_char2=np.array([char2Idx['PADDING_TOKEN']])
                    temp_char.append(temp_char2)
                    word_vector = [0] * nb_embedding_dims
                    temp_word.append(word_vector)
                else:
                    # char
                    temp_char2=[]
                    for char in word:
                        if char in char2Idx.keys():
                            temp_char2.append(char2Idx[char])
                        else:
                            temp_char2.append(char2Idx['UNKNOWN'])
                    temp_char2 = np.array(temp_char2)
                    temp_char.append(temp_char2)

                    # word
                    word_vector = ft.get_word_vector(word.lower())
                    # word_vector = ft.get_word_vector(word)
                    temp_word.append(word_vector)

            temp_char = pad_sequences(temp_char, 52)
            word_embeddings.append(temp_word)
            case_embeddings.append(temp_casing)
            char_embeddings.append(temp_char)
            temp_output = to_categorical(temp_output, len(label2Idx))
            output_labels.append(temp_output)

        return([np.array(word_embeddings), 
                np.array(case_embeddings), 
                np.array(char_embeddings)], 
               np.array(output_labels))

In [38]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=1

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=1


In [39]:
def get_model_3cnn():
    words_input = Input(shape=(None, nb_embedding_dims), dtype='float32', name='words_input')
    casing_input = Input(shape=(None,), dtype='int32', name='casing_input')
    casing = Embedding(output_dim=caseEmbeddings.shape[1], input_dim=caseEmbeddings.shape[0], weights=[caseEmbeddings], trainable=False, name = 'case_embed')(casing_input)
    character_input=Input(shape=(None,52,),name='char_input')
    embed_char_out=TimeDistributed(Embedding(len(char2Idx),32,embeddings_initializer=RandomUniform(minval=-0.5, maxval=0.5)), name='char_embedding')(character_input)
    kernel_sizes = (3, 4, 5)
    conv_blocks = []
    for sz in kernel_sizes:
        conv = TimeDistributed(Conv1D(
                             kernel_size=sz,
                             filters=32,
                             padding="same",
                             activation="relu",
                             strides=1,
                             name='charcnn_' + str(sz)))(embed_char_out)
        conv = TimeDistributed(MaxPooling1D(52, name = 'charcnn_maxpool'))(conv)
        conv = TimeDistributed(Flatten())(conv)
        conv_blocks.append(conv)
    output = concatenate([words_input, casing, conv_blocks[0], conv_blocks[1], conv_blocks[2]])
    output = Bidirectional(LSTM(200, return_sequences=True, dropout=0.50, recurrent_dropout=0.5, name='token_lstm'))(output)
    output = TimeDistributed(Dense(len(label2Idx), name = 'token_dense'))(output)
    crf = CRF(len(label2Idx))
    output = crf(output)
    model = Model(inputs=[words_input, casing_input, character_input], outputs=[output])
    model.compile(loss=crf.loss_function, optimizer='nadam', metrics=[crf.accuracy])
    model.summary()
    return(model)

nb_char_embedding_dims = 52
def get_model_lstm():
    words_input = Input(shape=(None, nb_embedding_dims), dtype='float32', name='words_input')
    casing_input = Input(shape=(None,), dtype='int32', name='casing_input')
    casing = Embedding(output_dim=caseEmbeddings.shape[1], input_dim=caseEmbeddings.shape[0], weights=[caseEmbeddings], trainable=False, name = 'case_embed')(casing_input)
    character_input=Input(shape=(None,nb_char_embedding_dims,),name='char_input')
    embed_char_out=TimeDistributed(Embedding(len(char2Idx),32,embeddings_initializer=RandomUniform(minval=-0.5, maxval=0.5)), name='char_embedding')(character_input)
    char_lstm = TimeDistributed(Bidirectional(LSTM(50)))(embed_char_out)
    output = concatenate([words_input, casing, char_lstm])
    output = Bidirectional(LSTM(200, return_sequences=True, dropout=0.50, recurrent_dropout=0.5))(output)
    output = TimeDistributed(Dense(len(label2Idx)))(output)
    crf = CRF(len(label2Idx))
    output = crf(output)
    model = Model(inputs=[words_input, casing_input, character_input], outputs=[output])
    model.compile(loss=crf.loss_function, optimizer='nadam', metrics=[crf.accuracy])
    model.summary()
    return(model)

class F1History(Callback):
    def on_train_begin(self, logs={}):
        self.acc = []
        self.f1_scores = []
        self.max_f1 = 0

    def on_epoch_end(self, epoch, logs={}):
        self.acc.append(logs.get('val_acc'))
        true_labels, pred_labels = predict_sequences(devSentences)
        pre, rec, f1 = compute_f1(pred_labels, true_labels, idx2Label)
        self.f1_scores.append(f1)
        if epoch > -1 and f1 > self.max_f1:
            print("\nNew maximum F1 score: " + str(f1) + " (before: " + str(self.max_f1) + ") Saving to " + tmp_model_filename)
            self.max_f1 = f1
            model.save(tmp_model_filename)

In [53]:
tmp_model_filename = 'wiki_POS_pretrained_LSTM-BiLSTM.h5'
checkpoint = ModelCheckpoint(tmp_model_filename, verbose=1, save_best_only = True, monitor = 'val_acc')
history = F1History()
model = get_model_3cnn()
model.fit_generator(
    NerSequence(trainSentences, shuffle_data=True, batch_size=128), 
    validation_data = NerSequence(devSentences, batch_size=256), 
    epochs = 10, callbacks = [checkpoint]
)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
char_input (InputLayer)         (None, None, 52)     0                                            
__________________________________________________________________________________________________
char_embedding (TimeDistributed (None, None, 52, 32) 10560       char_input[0][0]                 
__________________________________________________________________________________________________
time_distributed_31 (TimeDistri (None, None, 52, 32) 3104        char_embedding[0][0]             
__________________________________________________________________________________________________
time_distributed_34 (TimeDistri (None, None, 52, 32) 4128        char_embedding[0][0]             
__________________________________________________________________________________________________
time_distr

In [54]:
print(history.acc)
print(history.f1_scores)

AttributeError: 'F1History' object has no attribute 'acc'

In [55]:
model.save('wiki_POS_pretrained_LSTM-BiLSTM_10epochs.h5')

In [ ]:
# model.layers.pop()
# model.summary()

In [56]:
indexMappings = json.load(open("../models/final_model_germeval.indexes", "r"))
idx2Label = {int(k):v for k,v in indexMappings[0].items()}
label2Idx = indexMappings[1]
char2Idx = indexMappings[2]
case2Idx = indexMappings[3]

In [57]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
char_input (InputLayer)         (None, None, 52)     0                                            
__________________________________________________________________________________________________
char_embedding (TimeDistributed (None, None, 52, 32) 10560       char_input[0][0]                 
__________________________________________________________________________________________________
time_distributed_31 (TimeDistri (None, None, 52, 32) 3104        char_embedding[0][0]             
__________________________________________________________________________________________________
time_distributed_34 (TimeDistri (None, None, 52, 32) 4128        char_embedding[0][0]             
__________________________________________________________________________________________________
time_distr

In [62]:
model2 = get_model_3cnn()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
char_input (InputLayer)         (None, None, 52)     0                                            
__________________________________________________________________________________________________
char_embedding (TimeDistributed (None, None, 52, 32) 10560       char_input[0][0]                 
__________________________________________________________________________________________________
time_distributed_43 (TimeDistri (None, None, 52, 32) 3104        char_embedding[0][0]             
__________________________________________________________________________________________________
time_distributed_46 (TimeDistri (None, None, 52, 32) 4128        char_embedding[0][0]             
__________________________________________________________________________________________________
time_distr

In [65]:
for l in model.layers:
    print(l.name)
print('---')
for l in model2.layers:
    print(l.name)

char_input
char_embedding
time_distributed_31
time_distributed_34
time_distributed_37
casing_input
time_distributed_32
time_distributed_35
time_distributed_38
words_input
case_embed
time_distributed_33
time_distributed_36
time_distributed_39
concatenate_4
bidirectional_4
time_distributed_40
crf_4
---
char_input
char_embedding
time_distributed_43
time_distributed_46
time_distributed_49
casing_input
time_distributed_44
time_distributed_47
time_distributed_50
words_input
case_embed
time_distributed_45
time_distributed_48
time_distributed_51
concatenate_6
bidirectional_7
time_distributed_52
crf_6


In [66]:
model2.load_weights('wiki_POS_pretrained_LSTM-BiLSTM.h5', by_name=True)